In [ ]:
import os
import xml.etree.ElementTree as ET

def add_lane_names_in_files(folder_path):
    # Traverse the folder and process each XML file
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".xml"):
                file_path = os.path.join(subdir, file)
                add_lane_name_if_missing(file_path)

def add_lane_name_if_missing(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Find all <process> elements
    namespaces = {
        'bpmn': 'http://www.omg.org/spec/BPMN/20100524/MODEL'
    }
    
    processes = root.findall('.//bpmn:process', namespaces)

    # Flag to track if we made changes to the file
    changes_made = False

    # Loop through each <process> element
    for process in processes:
        # Get the name of the process
        process_name = process.attrib.get('name', None)
        
        if not process_name:
            continue  # Skip if no process name is available

        # Find all lanes in the current process's laneSet
        lanes = process.findall('.//bpmn:lane', namespaces)

        # Check each lane element for missing 'name' attribute
        for lane in lanes:
            if 'name' not in lane.attrib:
                # If the name attribute is missing, set it to the process name
                lane.set('name', process_name)
                changes_made = True

    # If changes were made, write back the updated XML to the file
    if changes_made:
        tree.write(file_path, encoding="UTF-8", xml_declaration=True)
        print(f"Updated file: {file_path}")

# Example usage
folder_containing_xml_files = "/path/to/your/folder"
add_lane_names_in_files(folder_containing_xml_files)
